In [14]:
import subprocess
import os
from LAMMPS_gen_in_file import * 
from LAMMPS_data_preprocess import * 
from LAMMPS_analysis import *
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [15]:
def run_lammps_simulation(velocity, temperature):
    dump_dir = "./runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    dump_dir_ = "runs/test_" + str(test_num) + "/vel_" + str(velocity) + "_temp_" + str(temperature)
    
    temp_ts = 0.03 * 0.02 / velocity
    
    try:
        os.mkdir("./runs")
    except FileExistsError:
        print('Folder already exist.')
        
    try:
        os.mkdir("./runs/test_" + str(1))
    except FileExistsError:
        print('Folder already exist.')

    generate_in_file(velocity, temperature, dump_dir=dump_dir, output_file=dump_dir_ + "/in.perforation", 
                     iterations=[5000, 5000, 0, 70000], ts = [0.1, 0.1, temp_ts])
    
    os.system(f'cp C60.data {dump_dir_}/C60.data')
    
    prepare_data(graphene_size=[20, 20], input_c60_file=f'{dump_dir_}/C60.data', 
                 output_c60_centered=f'{dump_dir_}/centered_C60.data', 
                 output_graphene_centered=f'{dump_dir_}/centered_graphene.data', 
                 output_file=f'{dump_dir_}/combined.data', 
                 output_graphene=f'{dump_dir_}/graphene.data', 
                 z_graphene=20, z_c60=5)

    lammps_path = "/home/korotnev/Documents/LAMMPS/lammps-29Aug2024/src/lmp_serial"  
    script_path = f'{dump_dir_}/in.perforation'

    try:
        result = subprocess.run([lammps_path, '-in', script_path], check=True, 
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"LAMMPS simulation for velocity {velocity} started successfully!")
        #print(result.stdout.decode())  
        #print(result.stderr.decode())  
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running LAMMPS for velocity {velocity}: {e}")
        
    df_graphene = get_dataset(get_files(dump_dir_, el='gr'))
    df_c60 = get_dataset(get_files(dump_dir_, el='c60'))
    
    get_plots_coords_velocities(df_graphene, name=dump_dir_ + '/graphene')
    get_plots_coords_velocities(df_c60, name=dump_dir_ + '/c60')
    
    print(f'Фуллерен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken_c60_coords(df_c60, z = 25)}')
    print(f'Графен при Температуре {temperature}, скорости {velocity} : сломан : {is_broken_graphene(df_graphene, alpha = 0.03)}')
    return is_broken_c60_coords(df_c60, z = 25)

In [16]:
def find_velocity(v_min, v_max, temp, tol = 0.1):
    print(f"Starting new velocity search with params\n temp {temp} \n v_min {v_min} \n v_max {v_max} \n")
    if 2 * (v_max - v_min) / (v_max + v_min) < tol:
        print(f"Solution has found, temp {temp}, velocity {(v_max + v_min) / 2}\n")
        return (v_max + v_min) / 2
    
    bool_ = run_lammps_simulation((v_max + v_min) / 2, temp)
    
    if bool_:
        return find_velocity(v_min, (v_max + v_min) / 2, temp, tol=tol)
    else:
        return find_velocity((v_max + v_min) / 2, v_max, temp, tol=tol)

In [17]:
def save_results_to_csv(file_path, velocity, temperature):
    data = {"Temperature": [temperature], "Velocity": [velocity]}
    df = pd.DataFrame(data)

    try:
        existing_df = pd.read_csv(file_path)
        new_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        new_df = df
        
    new_df.to_csv(file_path, index=False)

In [18]:
def start_search(v_min, v_max, temp, tol=0.01):
    result_file = "./runs/test_" + str(test_num) +"/results.csv" 

    b_min = run_lammps_simulation(v_min, temp)
    b_max = run_lammps_simulation(v_max, temp)

    if (b_min and b_max) or (not b_min and not b_max):
        return 0

    velocity = find_velocity(v_min, v_max, temp, tol=tol)

    # Сохранение результата в CSV
    save_results_to_csv(result_file, velocity, temp)

    return velocity

In [ ]:
test_num = 1

#velocities = [0.05, 0.1, 0.5, 1.]
temperatures = [200, 225, 250, 275, 300, 325, 350, 375, 400]

with ProcessPoolExecutor() as executor:
    futures = [executor.submit(start_search, 0.01, 1.0, temperature) for temperature in temperatures]
    
    for future in futures:
        future.result()

Folder already exist.Folder already exist.Folder already exist.Folder already exist.Folder already exist.Folder already exist.Folder already exist.


Folder already exist.



Folder already exist.
In File generated.
Folder already exist.Folder already exist.Folder already exist.Folder already exist.Folder already exist.
Folder already exist.






In File generated.
In File generated.
In File generated.
In File generated.
In File generated.
In File generated.


In File generated.





Preparing data started.
Preparing data started.
Preparing data started.

Preparing data started.
Preparing data started.
Preparing data started.
Preparing data started.






Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_200/graphene.data.Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_275/graphene.data.Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_250/graphene.data.Graphene structu


Data prepared
LAMMPS simulation for velocity 1.0 started successfully!
Фуллерен при Температуре 325, скорости 1.0 : сломан : True
Графен при Температуре 325, скорости 1.0 : сломан : True
Starting new velocity search with params
 temp 325 
 v_min 0.01 
 v_max 1.0 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.505_temp_325/graphene.data.
Graphene has generated. Centering data started.

LAMMPS simulation for velocity 1.0 started successfully!
Data centered. Merging data started.

Data prepared
Фуллерен при Температуре 350, скорости 1.0 : сломан : True
Графен при Температуре 350, скорости 1.0 : сломан : True
Starting new velocity search with params
 temp 350 
 v_min 0.01 
 v_max 1.0 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.505_temp_350/graphene.data.
Grap

/home/korotnev/Documents/LAMMPS/LAMMPS_fullerene_graphene/LAMMPS_analysis.py:121: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 8))


Фуллерен при Температуре 300, скорости 0.2575 : сломан : True
Графен при Температуре 300, скорости 0.2575 : сломан : True
Starting new velocity search with params
 temp 300 
 v_min 0.01 
 v_max 0.2575 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.13375_temp_300/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
LAMMPS simulation for velocity 0.2575 started successfully!


/home/korotnev/Documents/LAMMPS/LAMMPS_fullerene_graphene/LAMMPS_analysis.py:121: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 8))


...

LAMMPS simulation for velocity 0.07187500000000001 started successfully!
Фуллерен при Температуре 225, скорости 0.07187500000000001 : сломан : True
Графен при Температуре 225, скорости 0.07187500000000001 : сломан : True
Starting new velocity search with params
 temp 225 
 v_min 0.01 
 v_max 0.07187500000000001 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.0409375_temp_225/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
Фуллерен при Температуре 275, скорости 0.07187500000000001 : сломан : True
Графен при Температуре 275, скорости 0.07187500000000001 : сломан : True
Starting new velocity search with params
 temp 275 
 v_min 0.01 
 v_max 0.07187500000000001 

Folder already exist.
Folder already exist.
In File generated.

LAMMPS simulation for velocity 0.07187500000000001 started successfully!Preparing dat

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.064140625_temp_325/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
LAMMPS simulation for velocity 0.056406250000000005 started successfully!
Фуллерен при Температуре 300, скорости 0.056406250000000005 : сломан : False
Графен при Температуре 300, скорости 0.056406250000000005 : сломан : False
Starting new velocity search with params
 temp 300 
 v_min 0.056406250000000005 
 v_max 0.07187500000000001 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.064140625_temp_300/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
LAMMPS simulation for velocity 0.056406250000000005 started successfully!
LAMMPS simulation for velocity 0.056406250000000005 started successfully!
LAMMPS simula


Data prepared
LAMMPS simulation for velocity 0.064140625 started successfully!
Фуллерен при Температуре 375, скорости 0.064140625 : сломан : True
Графен при Температуре 375, скорости 0.064140625 : сломан : False
Starting new velocity search with params
 temp 375 
 v_min 0.056406250000000005 
 v_max 0.064140625 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.060273437500000006_temp_375/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
LAMMPS simulation for velocity 0.060273437500000006 started successfully!
Фуллерен при Температуре 325, скорости 0.060273437500000006 : сломан : True
Графен при Температуре 325, скорости 0.060273437500000006 : сломан : False
Starting new velocity search with params
 temp 325 
 v_min 0.056406250000000005 
 v_max 0.060273437500000006 

Folder already exist.
Folder already exist.



Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.059306640625000004_temp_275/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
Фуллерен при Температуре 200, скорости 0.05833984375000001 : сломан : True
Графен при Температуре 200, скорости 0.05833984375000001 : сломан : False
Starting new velocity search with params
 temp 200 
 v_min 0.056406250000000005 
 v_max 0.05833984375000001 

Folder already exist.
Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.05737304687500001_temp_200/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
Фуллерен при Температуре 350, скорости 0.05833984375000001 : сломан : True
Графен при Температуре 350, скорости 0.05833984375000001 : сломан : False
Starting new velocity search with params
 temp

Folder already exist.
In File generated.

Preparing data started.

Graphene structure with 270 atoms has been written to runs/test_1/vel_0.01_temp_400/graphene.data.
Graphene has generated. Centering data started.

Data centered. Merging data started.

Data prepared
LAMMPS simulation for velocity 0.05979003906250001 started successfully!
Фуллерен при Температуре 225, скорости 0.05979003906250001 : сломан : False
Графен при Температуре 225, скорости 0.05979003906250001 : сломан : False
Starting new velocity search with params
 temp 225 
 v_min 0.05979003906250001 
 v_max 0.060273437500000006 

Solution has found, temp 225, velocity 0.06003173828125001

LAMMPS simulation for velocity 0.057856445312500004 started successfully!
Фуллерен при Температуре 200, скорости 0.057856445312500004 : сломан : False
Графен при Температуре 200, скорости 0.057856445312500004 : сломан : False
Starting new velocity search with params
 temp 200 
 v_min 0.057856445312500004 
 v_max 0.05833984375000001 

Solu